In [206]:
import re
import pandas as pd
import geonamescache
from unidecode import unidecode

gc = geonamescache.GeonamesCache()
cities = gc.get_cities()
# print(cities)

black_lst = ["Can", "Come", "Of"]
city_dict_regexp = {}
city_lst = []
city_lst_regexp = []
city_str = ""
df_data = []

items = cities.items()
for item in items:
    city = item[1]['name']
    countrycode = item[1]['countrycode']
    latitude = item[1]['latitude']
    longitude = item[1]['longitude']
    if city not in black_lst:
        city = unidecode(city)
        city_dict_regexp[city] = [countrycode, latitude, longitude]

city_lst_regexp = [*city_dict_regexp]
city_lst_regexp.sort(key = len)  
city_lst_regexp.reverse()

city_dict_regexp = {k.lower(): v for k, v in city_dict_regexp.items()}

city_str = '|'.join(city_lst_regexp)
city_str = "\\b(" + city_str + ")\\b"

regexp = re.compile(r"(?P<city>{cities})".format(cities=city_str), re.I)


In [207]:
count = 0
cnt_ok = 0
cnt_nok = 0

country_dict = gc.get_countries()

with open("data/headlines.txt", "r") as fn:
    lines = fn.readlines()
    for line in lines:
        count = count + 1
        line = line.strip()
        result = regexp.search(line)
        if result != None:
            cnt_ok += 1
            city = result.group('city')
            city_hlp = city.lower()
            
            if city_hlp in city_dict_regexp:
                cc = city_dict[city_hlp][0]
                country = country_dict[cc]['name']
                latitude = city_dict_regexp[city_hlp][1]
                longitude = city_dict_regexp[city_hlp][2]
                # print(cnt_ok, "--", line, "--", city, "--", latitude, "--", longitude, "--", cc, "--", country)
                df_data.append((line, city, latitude, longitude, cc, country))
   
            else:
                None
                # print(count, "DBG-002:", "--", city, "not found in dictionary")
        else:
            cnt_nok +=1
            # print(count, "DBG-001:", "--", line, "not found in headlines")


In [208]:
df = pd.DataFrame(df_data, columns =['headline', 'city', 'latitude', 'longitude', 'countrycode', 'country'])
print("Nr of entries in data frame", df.shape[0])
df.to_csv("output.csv", index=False, header=True)
print(df.iloc[:10])

Nr of entries in data frame 613
                                  headline           city  latitude  \
0                 Zika Outbreak Hits Miami          Miami  25.77427   
1          Could Zika Reach New York City?  New York City  40.71427   
2        First Case of Zika in Miami Beach    Miami Beach  25.79065   
3  Mystery Virus Spreads in Recife, Brazil         Recife  -8.05389   
4  Dallas man comes down with case of Zika         Dallas  44.91928   
5        Trinidad confirms first Zika case       Trinidad -33.51650   
6   Zika Concerns are Spreading in Houston        Houston  29.76328   
7    Geneve Scientists Battle to Find Cure         Geneve  46.20222   
8    The CDC in Atlanta is Growing Worried        Atlanta  33.74900   
9       Zika Infested Monkeys in Sao Paulo      Sao Paulo -23.54750   

   longitude countrycode        country  
0  -80.19366          US  United States  
1  -74.00597          US  United States  
2  -80.13005          US  United States  
3  -34.88111      

In [209]:
print("Total nr of headlines:",count)   
print("Nr OK",cnt_ok)   
print("Nr NOK",cnt_nok)   

Total nr of headlines: 650
Nr OK 613
Nr NOK 37


In [210]:
test = gc.get_cities_by_name('San Bernardino')
test2 = gc.get_cities_by_name('Ho Chi Minh City')

In [211]:
print(test)

[{'5391710': {'geonameid': 5391710, 'name': 'San Bernardino', 'latitude': 34.10834, 'longitude': -117.28977, 'countrycode': 'US', 'population': 216108, 'timezone': 'America/Los_Angeles', 'admin1code': 'CA'}}]


In [212]:
print(test2)

[{'1566083': {'geonameid': 1566083, 'name': 'Ho Chi Minh City', 'latitude': 10.82302, 'longitude': 106.62965, 'countrycode': 'VN', 'population': 3467331, 'timezone': 'Asia/Ho_Chi_Minh', 'admin1code': '20'}}]


In [213]:
test3 = gc.get_cities_by_name('Davos')
print(test3)

[]
